# Salary Prediction Project

This notebook covers the end-to-end process of building a machine learning model to predict salary brackets.

In [5]:
! pip install seaborn matplotlib scikit-learn joblib 

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached matplotlib-3.10.8-cp311-cp311-win_amd64.whl.metadata (52 kB)
  Using cached scikit_learn-1.8.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached contourpy-1.3.3-cp311-cp311-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.61.1-cp311-cp311-win_amd64.whl.metadata (116 kB)
  Using cached kiwisolver-1.4.9-cp311-cp311-win_amd64.whl.metadata (6.4 kB)
  Using cached pillow-12.1.0-cp311-cp311-win_amd64.whl.metadata (9.0 kB)
  Using cached pyparsing-3.3.2-py3-none-any.whl.metadata (5.8 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached matplotlib-3.10.8-cp311-cp311-win_amd64.whl (8.1 MB)
Using cached scikit_learn-1.8.0-cp311-cp311-win_amd64.whl (8.1 MB)
Using cached joblib-1.5.3-py3-none-any.whl (309 kB)
Using cached contourpy-1.3.3-cp311-cp311-win_amd64

In [6]:
! pip install pandas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import joblib
import os

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scikit_learn-1.8.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached pillow-12.1.0-cp311-cp311-win_amd64.whl.metadata (9.0 kB)
  Using cached scipy-1.17.0-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached altair-6.0.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-6.2.4-py3-none-any.whl.metadata (5.6 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4

  You can safely remove it manually.
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'd:\\Salary estimation use KNN algoritham\\.venv\\Lib\\site-packages\\matplotlib\\_c_internal_utils.cp311-win_amd64.pyd'
Check the permissions.



KeyboardInterrupt: 

## 1. Load Dataset

In [ ]:
print("Loading dataset...")
files = [f for f in os.listdir('.') if f.endswith('.csv')]
if not files:
    raise FileNotFoundError("No CSV file found in the current directory.")
csv_file = files[0] # Auto-detect
print(f"Detected file: {csv_file}")
df = pd.read_csv(csv_file)

## 2. Exploratory Data Analysis (EDA)

In [ ]:
print("\n--- Dataset Info ---")
print(df.info())
print("\n--- Dataset Shape ---")
print(df.shape)
print("\n--- First 5 Rows ---")
print(df.head())

## 3. Handle Missing Values

In [ ]:
# Impute numerical with median, categorical with mode
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].fillna(df[col].mode()[0])
    else:
        df[col] = df[col].fillna(df[col].median())

## 4. Encode Target Variable

In [ ]:
# <=50K -> 0, >50K -> 1
# Check unique values to ensure correct mapping
print(f"\nUnique values in income before encoding: {df['income'].unique()}")
df['income'] = df['income'].apply(lambda x: 1 if '>50K' in str(x) else 0)
print(f"Unique values in income after encoding: {df['income'].unique()}")

## 5. Feature Selection and Splitting

In [ ]:
features = ['age', 'education.num', 'capital.gain', 'hours.per.week']
X = df[features]
y = df['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## 6. Feature Scaling

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 7. Model Training & Comparison

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(random_state=42)
}

### Find Best K for KNN

In [ ]:
print("\n--- Finding Best K for KNN ---")
k_values = [3, 5, 7, 9]
best_k = 3
best_k_acc = 0
knn_accuracies = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    score = knn.score(X_test_scaled, y_test)
    knn_accuracies.append(score)
    print(f"K={k}, Accuracy={score:.4f}")
    if score > best_k_acc:
        best_k_acc = score
        best_k = k

print(f"Best K selected: {best_k}")
models[f"KNN (k={best_k})"] = KNeighborsClassifier(n_neighbors=best_k)

## 8. Model Evaluation

In [ ]:
results = {}
best_model_name = ""
best_model_score = 0
best_model_obj = None

print("\n--- Model Evaluation ---")
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    
    results[name] = {
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1": f1,
        "Matrix": cm
    }
    
    print(f"\nModel: {name}")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("Confusion Matrix:\n", cm)
    
    if acc > best_model_score:
        best_model_score = acc
        best_model_name = name
        best_model_obj = model

print(f"\n🏆 Best Model: {best_model_name} with Accuracy check {best_model_score:.4f}")

## 9. Save Best Model and Scaler

In [ ]:
print("\nSaving best model and scaler...")
joblib.dump(best_model_obj, 'salary_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
print("Saved 'salary_model.pkl' and 'scaler.pkl'")

## 10. Visualization

In [ ]:
# Class Distribution
plt.figure(figsize=(6, 4))
sns.countplot(x=y)
plt.title("Income Class Distribution")
plt.show()

# Correlation Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(df[features + ['income']].corr(), annot=True, cmap='coolwarm')
plt.title("Feature Correlation Heatmap")
plt.show()

# KNN Accuracy Plot
plt.figure(figsize=(6, 4))
plt.plot(k_values, knn_accuracies, marker='o')
plt.title("KNN Accuracy vs K Value")
plt.xlabel("K")
plt.ylabel("Accuracy")
plt.show()

# Confusion Matrix Heatmap (Best Model)
plt.figure(figsize=(6, 5))
sns.heatmap(results[best_model_name]["Matrix"], annot=True, fmt='d', cmap='Blues')
plt.title(f"Confusion Matrix - {best_model_name}")
plt.show()